In [1]:
import pandas as pd

In [1]:
# Method to combine two CSV files together using Pandas merge calls
def main(file1, file2): # File1 should probably be the CD_CSV file mostly
    f1 = pd.read_csv(file1, sep='\t', encoding='utf-8')
    df_file1 = pd.DataFrame(f1)
    df_file2 = pd.DataFrame(pd.read_csv(file2, sep='\t', encoding='utf-8'))

    # print(df_file1['County'].head(5))
    if file2 == 'Fully_Cleaned_AEA_Data':
        # By making this new DataFrame I cut out the excess index column from the second file.
        stats_from_file_two = pd.DataFrame(data=[df_file2['County'], df_file2['Higher Degree'], df_file2['H.S Diploma'], df_file2['No H.S Diploma']]).transpose()
        df_output = pd.merge(df_file1, stats_from_file_two, on='County')
        df_output.to_csv('CD_and_Statistical_Atlas_Data_Combined', sep='\t', encoding='utf-8')
    elif file2 == 'Fully_Cleaned_SS_Data':
        summarize_school_scores(df_file2)
        # Make new data frame to remove excess index column
        # school_file_two = pd.DataFrame(data=[df_file2['County'], df_file2['Higher Degree'], df_file2['H.S Diploma'], df_file2['No H.S Diploma']]).transpose()
        # df_output = pd.merge(df_file1, df_file2, on='County')
    # print(df_output.head(5))

    # df_output.to_csv('CD_SA_SS_Data_Combined', sep='\t', encoding='utf-8')

In [ ]:
# Method to summarize all school score data into one score per county
def summarize_school_scores(data):
    advancedgroup = data['% Advanced'].groupby(data['County'])
    proficientgroup = data['% Proficient'].groupby(data['County'])
    basicgroup = data['% Basic'].groupby(data['County'])
    belowbgroup = data['% Advanced'].groupby(data['County'])

    print(advancedgroup.head(5))

In [ ]:
# Make sure to update what files you are passing in for combination. At this point,
# CD_and_Statistical_Atlas_Data_Combined is the master file currently
if __name__ == '__main__':
    main('CD_and_Statistical_Atlas_Data_Combined', 'Fully_Cleaned_SS_Data')